## MCMC Sampling

This notebook 

In [4]:
import os
import glob
import numpy as np
import matplotlib
import matplotlib.gridspec as gridspec
from matplotlib import pyplot as plt
from getdist import plots

# from mockfactory import Catalog
from cosmoprimo.fiducial import DESI
from desilike.theories.galaxy_clustering import FOLPSTracerPowerSpectrumMultipoles, FOLPSRCTracerPowerSpectrumMultipoles
from desilike.observables.galaxy_clustering import TracerPowerSpectrumMultipolesObservable
from desilike.emulators import EmulatedCalculator, Emulator, TaylorEmulatorEngine
from desilike.likelihoods import ObservablesGaussianLikelihood
from desilike.samplers.emcee import EmceeSampler
from desilike.samples import Profiles, plotting, Chain
from desilike import setup_logging
setup_logging()  # for logging messages

plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['legend.fontsize'] = 14
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [5]:
# vairables for targets
tracer = "ELG_LOPnotqso" 
region = "SGC"  # NGC or SGC
mocks_fn = os.path.join(os.environ['SCRATCH'], "mocks/")
pk_dir = os.path.join(os.environ['HOME'], "project_rc/main/data/pk/")
pk_run = os.path.join(os.environ['HOME'], "project_rc/main/desihub/pkrun.py")

In [6]:
# set the k bins
kmin     = 0.005
kmax     = 0.2
binning  = 0.005
k_ev     = np.arange(kmin, kmax+0.001, binning)
klim     = {ell*2: (kmin,kmax,binning) for ell in range(2)}

redshift = 1.0
cosmology = 'LCDM' #LCDM, nuCDM, wCDM
theory_model = 'FOLPS'  # TNS, FOLPS, FOLPSRC

emulator_fn = f'./results/emulators/emulator_{cosmology}_{theory_model}.npy'
chain_fn = f'./results/samples/{cosmology}/chain_{tracer}_{region}_z{redshift}.npy'

In [ ]:
if not os.path.exists(chain_fn):
    pk_fn = pk_dir+f'pkpoles_{tracer}_{region}_0.8_1.1_default_FKP_lin_thetacut0.05.npy'
    cov_dir = '/global/cfs/cdirs/desi/survey/catalogs/Y1/mocks/SecondGenMocks/EZmock/desipipe/v1/ffa/2pt/'
    cov_fn = f'pkpoles_ELG_LOP_{region}_z0.8-1.1_default_FKP_lin_nran10_cellsize6_boxsize9000.npy'

    cov_fns = []
    for i in range(100, 200):
        mock_dir = f'mock{i}/pk/'
        cov_fns.extend(glob.glob(os.path.join(cov_dir,  mock_dir, cov_fn), recursive=True))
    
    if theory_model == 'FOLPS':
        theory = FOLPSTracerPowerSpectrumMultipoles(pt=EmulatedCalculator.load(emulator_fn))
    if theory_model == 'FOLPSRC':
        theory = FOLPSRCTracerPowerSpectrumMultipoles(pt=EmulatedCalculator.load(emulator_fn))  
    
    observable = TracerPowerSpectrumMultipolesObservable(data= pk_fn, 
                                                        klim=klim, 
                                                        covariance = cov_fns,)
    likelihood = ObservablesGaussianLikelihood(observable)
    likelihood()
    sampler = EmceeSampler(likelihood, seed=42, nwalkers=120, save_fn =chain_fn)
    sampler.run(check={'max_eigen_gr': 0.05}, max_iterations = 9001) # save every 300 iterations

In [ ]:
burnin      = 0.60
slice_step  = 500

chain       = Chain.load(chain_fn).remove_burnin(burnin)[::slice_step]
# print(chain.to_stats(tablefmt='pretty'))
samples     = chain

params = ['h','omega_cdm','logA']

g = plots.get_subplot_plotter()
g.settings.fig_width_inch= 8
g.settings.legend_fontsize = 20
g.settings.axes_labelsize = 20
g.settings.axes_fontsize = 16
g.settings.figure_legend_frame = False

def plot_cosparams():
    plotting.plot_triangle(chain, title_limit=0, params = params,
                               legend_labels = [r'clean', r'1% catas', r'5% catas'], legend_loc= 'upper right',
                                # legend_labels=[],
                                smoothed=True, g=g)
    def set_true_values(catalogue, params):
        update_values = {
            'fiducial': {'h': 0.6711, 'omega_cdm': 0.1209, 'Omega_cdm': 0.2685, 'omega_b':0.02207,'logA': 3.0631, 'm_ncdm': 0.0, 'n_s':0.9624, 'w0_fld':-1.0},
            'Mnu_p': {'h': 0.6711, 'omega_cdm': 0.1198, 'Omega_cdm': 0.2661, 'omega_b':0.02207,'logA': 3.1247, 'm_ncdm': 0.1, 'n_s':0.9624, 'w0_fld':-1.0},
            'Mnu_ppp': {'h': 0.6711, 'omega_cdm': 0.1166, 'Omega_cdm': 0.2590, 'omega_b':0.02207,'logA': 3.3113, 'm_ncdm': 0.4, 'n_s':0.9624, 'w0_fld':-1.0}
        }
        if catalogue in update_values:
            truth_values = update_values[catalogue]
        return [truth_values[param] for param in params if param in truth_values]
    true_values = set_true_values(catalogue, params)
    for i in range(len(true_values)):
        for j in range(i+1):
            g.subplots[i,j].axvline(true_values[j], c = 'k', ls = ':', lw = 1.2)
            if i != j:
                g.subplots[i,j].axhline(true_values[i], c = 'k', ls = ':', lw = 1.2)